In [1]:
### PREREQUISTIES
### (many used only in one notebook...)

import os
from importlib import reload
import pandas as pd
import numpy as np
import pickle
import math
import random
import sys
import csv
import unicodedata
import requests
import re
import networkx as nx
from scipy import stats
import plotly
import kaleido

import nltk

import matplotlib.pyplot as plt
import seaborn as sns

# gensim parts
from gensim import corpora
from gensim import models

### scikit-learn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.decomposition import TruncatedSVD 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances

import plotly.graph_objects as go

import sddk

In [2]:
import textnet

In [25]:
reload(textnet)

<module 'textnet' from '/home/kasev/ECCE_AGT/scripts/textnet.py'>

In [4]:
### not neccessary for reading the data, just for exporting them to sciencedata.dk
conf = sddk.configure("SDAM_root", "648597@au.dk")

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ········
connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/


In [5]:
AGT = sddk.read_file("SDAM_data/AGT/AGT_lemmatized_20201127.json", "df", conf)
AGT.head(5)

,filename,author,title,wordcount,author_id,doc_id,raw_date,date_avr,date_probs,date_manual,provenience,tlg_epithet,clean_string,sentences,n_sentences,lemmatized_sentences
0,tlg0001.tlg001.perseus-grc2.xml,Apollonius Rhodius,Argonautica,38822,tlg0001,tlg0001.tlg001,3 B.C.,-2.5,{'-2.5': 1},-2.5,pagan,Epici/-ae,"ἀρχόμενος σέο, Φοῖβε, παλαιγενέων κλέα φωτῶν μ...","[ἀρχόμενος σέο, Φοῖβε, παλαιγενέων κλέα φωτῶν ...",3252,"[[ἄρχω, Φοῖβος, παλαιγενής, κλέος, φώς, μιμνήσ..."
1,tlg0003.tlg001.perseus-grc2.xml,Thucydides,The Peloponnesian War,150118,tlg0003,tlg0003.tlg001,5 B.C.,-4.5,{'-4.5': 1},-4.5,pagan,Historici/-ae,Θουκυδίδης Ἀθηναῖος ξυνέγραψε τὸν πόλεμον τῶν...,[Θουκυδίδης Ἀθηναῖος ξυνέγραψε τὸν πόλεμον τῶν...,6068,"[[θουκυδίδης, Ἀθηναῖος, συγγράφω, πόλεμος, Πελ..."
2,tlg0004.tlg001.perseus-grc1.xml,Diogenes Laertius,Lives of Eminent Philosophers,110763,tlg0004,tlg0004.tlg001,A.D. 3,2.5,{'2.5': 1},NaN,,Biographi,Τὸ τῆς φιλοσοφίας ἔργον ἔνιοί φασιν ἀπὸ βαρβάρ...,[Τὸ τῆς φιλοσοφίας ἔργον ἔνιοί φασιν ἀπὸ βαρβά...,10245,"[[φιλοσοφία, ἔργον, ἔνιοι, φημί, βάρβαρος, ἄρχ..."
3,tlg0005.tlg001.perseus-grc1.xml,Theocritus,Idylls,19200,tlg0005,tlg0005.tlg001,4-3 B.C.,-3.0,"{'-3.5': 0.5, '-2.5': 0.5}",NaN,,Bucolici,"̔Αδύ τι τὸ ψιθύρισμα καὶ ἁ πίτυς αἰπόλε τήνα,...","[Αδύ τι τὸ ψιθύρισμα καὶ ἁ πίτυς αἰπόλε τήνα, ...",1982,"[[ψιθύρισμα, πίτυς, αἰπόλος, τῆνος, πηγή, μελί..."
4,tlg0005.tlg002.perseus-grc1.xml,Theocritus,Epigrams,1734,tlg0005,tlg0005.tlg002,4-3 B.C.,-3.0,"{'-3.5': 0.5, '-2.5': 0.5}",NaN,,Bucolici,τὰ ῥόδα τὰ δροσόεντα καὶ ἁ κατάπυκνος ἐκείνα ἕ...,[τὰ ῥόδα τὰ δροσόεντα καὶ ἁ κατάπυκνος ἐκείνα ...,152,"[[ῥόδον, δροσόεντα, κατάπυκνος, ἐκεῖνος, ἕρπυλ..."


In [6]:
def flat_lemmata(sentences):
    lemmata = [word for sent in sentences for word in sent]
    return lemmata
AGT["lemmata"] = AGT["lemmatized_sentences"].apply(flat_lemmata)

AGT["lemmata_wordcount"] = AGT.apply(lambda row: len(row["lemmata"]), axis=1)
AGT["lemmata_wordcount"].sum() # previously we had 13925726, then 13713183, finally 14756899

14601227

In [7]:
def get_flat_sentences(series):
    sentences_list = [sent for doc in series.tolist() for sent in doc]
    return sentences_list

In [8]:
subcorpora_dict = {
    "Pagan (8th c. BCE - 4th c. CE)" : AGT[(AGT["provenience"]=="pagan") & (AGT["date_avr"] < 4)],
    "Pagan (1st - 4th c. CE)" : AGT[(AGT["date_avr"].between(0, 4)) & (AGT["provenience"]=="pagan")],
    "Septuagint" : AGT[AGT["author_id"]=="tlg0527"],
    "Philo of Alexandria" : AGT[AGT["author_id"]=="tlg0018"],
    "Titus Flavius Josephus" : AGT[AGT["author_id"]=="tlg0526"],
    "Paul of Tarsus" : AGT[AGT["author_id"]=="tlg0031paul"],
    "New Testament" : AGT[AGT["author_id"].str.startswith("tlg0031")],
    "Christian (1st - 4th c. CE)" : AGT[(AGT["date_avr"] < 4) & (AGT["provenience"]=="christian")]}

In [9]:
subselections = []
for key in subcorpora_dict.keys():
    subselection = subcorpora_dict[key]
    subselection_dict = {}
    wordcount = subselection["wordcount"].sum()
    doccount = len(subselection)
    sents = get_flat_sentences(subselection["lemmatized_sentences"])
    sents_len = len(sents)
    subselection_dict.update({"label" : key, "wordcount" : wordcount, "doccount" : doccount, "n_sentences": sents_len,"sentences" : sents})
    subselections.append(subselection_dict)
subselections_df = pd.DataFrame(subselections)
subselections_df

,label,wordcount,doccount,n_sentences,sentences
0,Pagan (8th c. BCE - 4th c. CE),12965219,774,1039409,"[[ἄρχω, Φοῖβος, παλαιγενής, κλέος, φώς, μιμνήσ..."
1,Pagan (1st - 4th c. CE),8692448,411,723956,"[[γεωγραφία, σόσσιε, σενεκίων, ἱστορικός, διαφ..."
2,Septuagint,799083,55,57856,"[[εἰμί, ποιέω, θεός, οὐρανός, γῆ], [γῆ, εἰμί, ..."
3,Philo of Alexandria,643011,31,82966,"[[ἄλλος, νομοθέτης, ἀκαλλώπιστος, γυμνός, νομι..."
4,Titus Flavius Josephus,469580,4,19707,"[[ἱστορία, συγγράφω, βούλομαι, αὐτός, ὁράω, σπ..."
5,Paul of Tarsus,24066,7,1632,"[[Παῦλος, δοῦλος, Ἰησοῦς, Χριστός, κλητός, ἀπό..."
6,New Testament,137787,27,8698,"[[βίβλος, γένεσις, Ἰησοῦς, Χριστός, υἱός, Δαυί..."
7,Christian (1st - 4th c. CE),2975644,131,291084,"[[βίβλος, γένεσις, Ἰησοῦς, Χριστός, υἱός, Δαυί..."


# term2term & term2doc matraces with sklearn - Paul as example

In [10]:
# let's explore Pa
docs = subselections_df.at[5, "sentences"]

In [11]:
len(docs)

1632

In [12]:
AGT_C = subcorpora_dict["Christian (1st - 4th c. CE)"]
len(AGT_C)

131

In [13]:
AGT_C.head(5)

,filename,author,title,wordcount,author_id,doc_id,raw_date,date_avr,date_probs,date_manual,provenience,tlg_epithet,clean_string,sentences,n_sentences,lemmatized_sentences,lemmata,lemmata_wordcount
385,tlg0031.tlg001.perseus-grc2.xml,None,New Testament - Matthew,18288,tlg0031a,tlg0031.tlg001,A.D. 1,0.5,{'0.5': 1},0.5,christian,[],ΒΙΒΛΟΣ γενέσεως Ἰησοῦ Χριστοῦ υἱοῦ Δαυεὶδ υἱο...,[ΒΙΒΛΟΣ γενέσεως Ἰησοῦ Χριστοῦ υἱοῦ Δαυεὶδ υἱο...,1276,"[[βίβλος, γένεσις, Ἰησοῦς, Χριστός, υἱός, Δαυί...","[βίβλος, γένεσις, Ἰησοῦς, Χριστός, υἱός, Δαυίδ...",8658
386,tlg0031.tlg002.perseus-grc2.xml,None,New Testament - Mark,11274,tlg0031b,tlg0031.tlg002,A.D. 1,0.5,{'0.5': 1},0.5,christian,[],ΑΡΧΗ τοῦ εὐαγγελίου Ἰησοῦ Χριστοῦ . Καθὼς γέγ...,"[ΑΡΧΗ τοῦ εὐαγγελίου Ἰησοῦ Χριστοῦ ., Καθὼς γέ...",790,"[[ἀρχή, εὐαγγέλιον, Ἰησοῦς, Χριστός], [γράφω, ...","[ἀρχή, εὐαγγέλιον, Ἰησοῦς, Χριστός, γράφω, ἠσα...",5298
387,tlg0031.tlg003.perseus-grc2.xml,Luke (the evangelist),New Testament - Luke,19458,tlg0031luke,tlg0031.tlg003,A.D. 1,0.5,{'0.5': 1},0.5,christian,[],ΕΠΕΙΔΗΠΕΡ ΠΟΛΛΟΙ ἐπεχείρησαν ἀνατάξασθαι διήγ...,[ΕΠΕΙΔΗΠΕΡ ΠΟΛΛΟΙ ἐπεχείρησαν ἀνατάξασθαι διήγ...,1274,"[[πολύς, ἐπιχειρέω, ἀνατάσσομαι, διήγησις, πλη...","[πολύς, ἐπιχειρέω, ἀνατάσσομαι, διήγησις, πληρ...",9195
388,tlg0031.tlg004.perseus-grc2.xml,Johnannine literature,New Testament - John,15590,tlg0031john,tlg0031.tlg004,A.D. 1,0.5,{'0.5': 1},0.5,christian,[],"ΕΝ ΑΡΧΗ ἦν ὁ λόγος, καὶ ὁ λόγος ἦν πρὸς τὸν θ...","[ΕΝ ΑΡΧΗ ἦν ὁ λόγος, καὶ ὁ λόγος ἦν πρὸς τὸν θ...",1164,"[[εἰμί, ἀρχή, εἰμί, λόγος, λόγος, εἰμί, θεός, ...","[εἰμί, ἀρχή, εἰμί, λόγος, λόγος, εἰμί, θεός, θ...",7060
389,tlg0031.tlg005.perseus-grc2.xml,Luke (the evangelist),New Testament - Acts,18406,tlg0031luke,tlg0031.tlg005,A.D. 1,0.5,{'0.5': 1},0.5,christian,[],"τὸν μὲν πρῶτον λόγον ἐποιησάμην περὶ πάντων, ...","[τὸν μὲν πρῶτον λόγον ἐποιησάμην περὶ πάντων, ...",960,"[[πρότερος, λόγος, ποιέω, πᾶς, Θεόφιλος, ἔρδω,...","[πρότερος, λόγος, ποιέω, πᾶς, Θεόφιλος, ἔρδω, ...",9096


In [17]:
author = AGT_C["author"][10]
author

'Paul of Tarsus'

In [18]:
sub_df = AGT_C[AGT_C["author_id"]=="tlg0031john"]
sub_df["date_probs"]

388    {'0.5': 1}
407    {'0.5': 1}
408    {'0.5': 1}
409    {'0.5': 1}
Name: date_probs, dtype: object

In [19]:
get_flat_sentences(sub_df["lemmatized_sentences"])

[['εἰμί',
  'ἀρχή',
  'εἰμί',
  'λόγος',
  'λόγος',
  'εἰμί',
  'θεός',
  'θεός',
  'εἰμί',
  'λόγος'],
 ['οὗτος', 'ἀρχή', 'θεός'],
 ['πᾶς', 'γίγνομαι', 'γίγνομαι', 'εἷς'],
 ['γίγνομαι', 'ζωή', 'εἰμί', 'ζωή', 'φώς', 'ἄνθρωπος'],
 ['φώς', 'σκοτία', 'φαίνω', 'σκοτία', 'καταλαμβάνω'],
 ['γίγνομαι', 'ἄνθρωπος', 'ἀποστέλλω', 'θεός', 'ὄνομα', 'ἰωάνης'],
 ['οὗτος',
  'ἔρχομαι',
  'μαρτυρία',
  'μαρτύρομαι',
  'φώς',
  'πᾶς',
  'πιστεύω',
  'αὐτοῦ.'],
 ['εἰμί', 'ἐκεῖνος', 'φώς', 'μαρτύρομαι', 'φώς'],
 ['φώς', 'ἀληθινός', 'φωτίζω', 'ἄνθρωπος', 'ἔρχομαι', 'κόσμος'],
 ['κόσμος', 'εἰμί', 'κόσμος', 'γίγνομαι', 'κόσμος', 'γιγνώσκω'],
 ['ἴδιος', 'ἔρχομαι', 'ἴδιος', 'παραλαμβάνω'],
 ['λαμβάνω',
  'δίδωμι',
  'ἐξουσία',
  'τέκνον',
  'θεός',
  'γίγνομαι',
  'πιστεύω',
  'ὄνομα',
  'αἷμα',
  'θέλημα',
  'σάρξ',
  'θέλημα',
  'ἀνήρ',
  'θεός',
  'γεννάω'],
 ['λόγος',
  'σάρξ',
  'γίγνομαι',
  'σκηνόω',
  'θεάομαι',
  'δόξα',
  'δόξα',
  'μονογενής',
  'πατήρ',
  'πλήρης',
  'χάρις',
  'ἀλήθεια'],
 [],
 [

In [20]:
probs_dict = {}
for d in sub_df["date_probs"]:
    probs_dict.update(d)
probs_dict

{'0.5': 1}

In [21]:
authors_dicts = []
for author_id in AGT_C["author_id"].unique():
    sub_df = AGT_C[AGT_C["author_id"]==author_id]
    authors_dict = {}
    authors_dict["author_id"] = author_id
    authors_dict["author"] = sub_df["author"][0]
    authors_dict["n_titles"] = len(sub_df)
    authors_dict["titles"] = sub_df["title"].tolist()
    authors_dict["filenames"] = sub_df["filename"].tolist()
    authors_dict["wordcount"] = sub_df["wordcount"].sum()
    authors_dict["lemmata_wordcount"] = sub_df["lemmata_wordcount"].sum()
    authors_dict["n_sentences"] = sub_df["n_sentences"].sum()
    probs_dict = {}
    for d in sub_df["date_probs"]:
        probs_dict.update(d)
    authors_dict["date_probs"] = probs_dict
    authors_dict["lemmata_lists"] = sub_df["lemmata"].tolist()
    authors_dict["lemmatized_sentences"] = get_flat_sentences(sub_df["lemmatized_sentences"])
    authors_dicts.append(authors_dict)
    
authors_df = pd.DataFrame(authors_dicts)
authors_df.head(5)

,author_id,author,n_titles,titles,filenames,wordcount,lemmata_wordcount,n_sentences,date_probs,lemmata_lists,lemmatized_sentences
0,tlg0031a,None,1,[New Testament - Matthew],[tlg0031.tlg001.perseus-grc2.xml],18288,8658,1276,{'0.5': 1},"[[βίβλος, γένεσις, Ἰησοῦς, Χριστός, υἱός, Δαυί...","[[βίβλος, γένεσις, Ἰησοῦς, Χριστός, υἱός, Δαυί..."
1,tlg0031b,None,1,[New Testament - Mark],[tlg0031.tlg002.perseus-grc2.xml],11274,5298,790,{'0.5': 1},"[[ἀρχή, εὐαγγέλιον, Ἰησοῦς, Χριστός, γράφω, ἠσ...","[[ἀρχή, εὐαγγέλιον, Ἰησοῦς, Χριστός], [γράφω, ..."
2,tlg0031luke,Luke (the evangelist),2,"[New Testament - Luke, New Testament - Acts]","[tlg0031.tlg003.perseus-grc2.xml, tlg0031.tlg0...",37864,18291,2234,{'0.5': 1},"[[πολύς, ἐπιχειρέω, ἀνατάσσομαι, διήγησις, πλη...","[[πολύς, ἐπιχειρέω, ἀνατάσσομαι, διήγησις, πλη..."
3,tlg0031john,Johnannine literature,4,"[New Testament - John, New Testament - 1 John,...","[tlg0031.tlg004.perseus-grc2.xml, tlg0031.tlg0...",18189,8221,1339,{'0.5': 1},"[[εἰμί, ἀρχή, εἰμί, λόγος, λόγος, εἰμί, θεός, ...","[[εἰμί, ἀρχή, εἰμί, λόγος, λόγος, εἰμί, θεός, ..."
4,tlg0031paul,Paul of Tarsus,7,"[New Testament - Romans, New Testament - 1 Cor...","[tlg0031.tlg006.perseus-grc2.xml, tlg0031.tlg0...",24066,11113,1632,{'0.5': 1},"[[Παῦλος, δοῦλος, Ἰησοῦς, Χριστός, κλητός, ἀπό...","[[Παῦλος, δοῦλος, Ἰησοῦς, Χριστός, κλητός, ἀπό..."


In [45]:
def ego_network_from_sentences(list_of_sentences):
    try:
        G = textnet.network_from_sentences(list_of_sentences)
    except:
        pass
    try:
        G_ego = textnet.construct_ego_network(G, "δικαιοσύνη", 30, reduced=False)
    except:
        G_ego = nx.Graph()
        G_ego.clear()
    return G_ego

def ego_network_from_lemmata(list_of_sentences):
    try:
        G = textnet.network_from_lemmata_lists(list_of_sentences)
    except:
        pass
    try:
        G_ego = textnet.construct_ego_network(G, "δικαιοσύνη", 30, reduced=False)
    except:
        G_ego = nx.Graph()
        G_ego.clear()
    return G_ego

In [49]:
authors_df["dikaiosyne_G_sents"] = authors_df["lemmatized_sentences"].apply(ego_network_from_sentences)

In [48]:
authors_df["dikaiosyne_G_lems"] = authors_df["lemmata_lists"].apply(ego_network_from_lemmata)

In [ ]:
authors_df.iloc[0][]

In [50]:
list(authors_df.iloc[0]["dikaiosyne_G_sents"].nodes())

['ἔχω',
 'οὐρανός',
 'ποιέω',
 'δίδωμι',
 'κύριος',
 'εἰμί',
 'ἡμέρα',
 'βασίλειον',
 'πᾶς',
 'προφήτης',
 'λέγω',
 'δικαιοσύνη',
 'γεννάω',
 'προσέρχομαι',
 'γίγνομαι',
 'μακάριος',
 'ἀκούω',
 'γῆ',
 'μαθητής',
 'οὗτος',
 'θεός',
 'βασίλεια',
 'ἀποκρίνω',
 'ὁράω',
 'βασιλεία',
 'πατήρ',
 'υἱός',
 'ἔρχομαι',
 'πληρόω',
 'ἄνθρωπος',
 'εἶπον']

In [51]:
authors_df["dik_G_sents_nodes"] = authors_df["dikaiosyne_G_sents"].apply(lambda cell: list(cell.nodes()))

In [53]:
authors_df["dik_G_lems_nodes"] = authors_df["dikaiosyne_G_lems"].apply(lambda cell: list(cell.nodes()))

In [54]:
def sentences_with_term(sentences, term, flatlist=False):
    sentences_with_term = [sen for sen in sentences if term in sen]
    if flatlist == True:
        flatlist = [word for sen in sentences_with_term for word in sen]
        return flatlist
    else: 
        return sentences_with_term
authors_df["dik_sents_flat"] = authors_df["lemmatized_sentences"].apply(lambda cell: sentences_with_term(cell, "δικαιοσύνη", flatlist=True))

In [63]:
def get_tf(lemmata):
    fd_list = list(nltk.FreqDist(lemmata).most_common())
    tf_list = [(tup[0], np.round(tup[1] / len(lemmata), 5)) for tup in fd_list]
    return tf_list

authors_df["dik_sents_TF"] = authors_df["dik_sents_flat"].apply(get_tf)

In [72]:
math.log(1000)

6.907755278982137

In [78]:
def term_idf(term, all_sentences):
    idf_raw =  len(all_sentences) / len([sent for sent in all_sentences if term in sent])
    idf_log = math.log(idf_raw)
    return idf_log

def get_tfidf(key_term_lemmata, all_sentences):
    fd_list = list(nltk.FreqDist(key_term_lemmata).most_common())
    tfidf_list = [(tup[0], np.round((tup[1] / len(key_term_lemmata) * term_idf(tup[0], all_sentences)), 5)) for tup in fd_list]
    return tfidf_list

authors_df["dik_sents_TFIDF"] = authors_df.apply(lambda x: get_tfidf(x["dik_sents_flat"], x["lemmatized_sentences"]), axis=1)

In [79]:
authors_df.head(5)

,author_id,author,n_titles,titles,filenames,wordcount,lemmata_wordcount,n_sentences,date_probs,lemmata_lists,lemmatized_sentences,dikaiosyne_G_sents,dikaiosyne_G_lems,dik_G_sents_nodes,dik_G_lems_nodes,dik_sents_flat,dik_sents_TF,dik_sents_TFIDF
0,tlg0031a,None,1,[New Testament - Matthew],[tlg0031.tlg001.perseus-grc2.xml],18288,8658,1276,{'0.5': 1},"[[βίβλος, γένεσις, Ἰησοῦς, Χριστός, υἱός, Δαυί...","[[βίβλος, γένεσις, Ἰησοῦς, Χριστός, υἱός, Δαυί...","(ἔχω, οὐρανός, ποιέω, δίδωμι, κύριος, εἰμί, ἡμ...","(εἶπον, Ἰησοῦς, υἱός, ἄνθρωπος, λέγω, κύριος, ...","[ἔχω, οὐρανός, ποιέω, δίδωμι, κύριος, εἰμί, ἡμ...","[εἶπον, Ἰησοῦς, υἱός, ἄνθρωπος, λέγω, κύριος, ...","[ἀποκρίνω, Ἰησοῦς, εἶπον, ἀφίημι, πρέπω, πληρό...","[(δικαιοσύνη, 0.15556), (πᾶς, 0.04444), (μακάρ...","[(δικαιοσύνη, 0.80976), (πᾶς, 0.13164), (μακάρ..."
1,tlg0031b,None,1,[New Testament - Mark],[tlg0031.tlg002.perseus-grc2.xml],11274,5298,790,{'0.5': 1},"[[ἀρχή, εὐαγγέλιον, Ἰησοῦς, Χριστός, γράφω, ἠσ...","[[ἀρχή, εὐαγγέλιον, Ἰησοῦς, Χριστός], [γράφω, ...",(),(),[],[],[],[],[]
2,tlg0031luke,Luke (the evangelist),2,"[New Testament - Luke, New Testament - Acts]","[tlg0031.tlg003.perseus-grc2.xml, tlg0031.tlg0...",37864,18291,2234,{'0.5': 1},"[[πολύς, ἐπιχειρέω, ἀνατάσσομαι, διήγησις, πλη...","[[πολύς, ἐπιχειρέω, ἀνατάσσομαι, διήγησις, πλη...","(ποιέω, δίδωμι, κύριος, εἰμί, τὶς, ἡμέρα, αὐτό...",(δικαιοσύνη),"[ποιέω, δίδωμι, κύριος, εἰμί, τὶς, ἡμέρα, αὐτό...",[δικαιοσύνη],"[Ζαχαρίας, πατήρ, πίμπλημι, πνεῦμα, ἅγιος, προ...","[(πᾶς, 0.04202), (δικαιοσύνη, 0.04202), (ἅγιος...","[(πᾶς, 0.11077), (δικαιοσύνη, 0.25639), (ἅγιος..."
3,tlg0031john,Johnannine literature,4,"[New Testament - John, New Testament - 1 John,...","[tlg0031.tlg004.perseus-grc2.xml, tlg0031.tlg0...",18189,8221,1339,{'0.5': 1},"[[εἰμί, ἀρχή, εἰμί, λόγος, λόγος, εἰμί, θεός, ...","[[εἰμί, ἀρχή, εἰμί, λόγος, λόγος, εἰμί, θεός, ...","(ἀγαπάω, κόσμος, πᾶς, μαθητής, ἔργον, οὗτος, π...","(λέγω, Ἰησοῦς, εἶπον, ἀποκρίνω, οὗτος, εἰμί, κ...","[ἀγαπάω, κόσμος, πᾶς, μαθητής, ἔργον, οὗτος, π...","[λέγω, Ἰησοῦς, εἶπον, ἀποκρίνω, οὗτος, εἰμί, κ...","[ἔρχομαι, ἐκεῖνος, ἐλέγχω, κόσμος, ἁμαρτία, δι...","[(δικαιοσύνη, 0.16667), (δίκαιος, 0.1), (εἰμί,...","[(δικαιοσύνη, 0.93171), (δίκαιος, 0.51202), (ε..."
4,tlg0031paul,Paul of Tarsus,7,"[New Testament - Romans, New Testament - 1 Cor...","[tlg0031.tlg006.perseus-grc2.xml, tlg0031.tlg0...",24066,11113,1632,{'0.5': 1},"[[Παῦλος, δοῦλος, Ἰησοῦς, Χριστός, κλητός, ἀπό...","[[Παῦλος, δοῦλος, Ἰησοῦς, Χριστός, κλητός, ἀπό...","(χριστός, ἔχω, νομός, δίδωμι, κύριος, ἔθνος, ε...","(Χριστός, Ἰησοῦς, κύριος, εἰμί, θεός, χάρις, π...","[χριστός, ἔχω, νομός, δίδωμι, κύριος, ἔθνος, ε...","[Χριστός, Ἰησοῦς, κύριος, εἰμί, θεός, χάρις, π...","[δικαιοσύνη, θεός, ἀποκαλύπτω, πίστις, πίστις,...","[(δικαιοσύνη, 0.11163), (θεός, 0.04186), (πίστ...","[(δικαιοσύνη, 0.41399), (θεός, 0.06833), (πίστ..."


In [60]:
authors_df

,author_id,author,n_titles,titles,filenames,wordcount,lemmata_wordcount,n_sentences,date_probs,lemmata_lists,lemmatized_sentences,dikaiosyne_G_sents,dikaiosyne_G_lems,dik_G_sents_nodes,dik_G_lems_nodes,dik_sents_flat,dik_sents_TF
0,tlg0031a,None,1,[New Testament - Matthew],[tlg0031.tlg001.perseus-grc2.xml],18288,8658,1276,{'0.5': 1},"[[βίβλος, γένεσις, Ἰησοῦς, Χριστός, υἱός, Δαυί...","[[βίβλος, γένεσις, Ἰησοῦς, Χριστός, υἱός, Δαυί...","(ἔχω, οὐρανός, ποιέω, δίδωμι, κύριος, εἰμί, ἡμ...","(εἶπον, Ἰησοῦς, υἱός, ἄνθρωπος, λέγω, κύριος, ...","[ἔχω, οὐρανός, ποιέω, δίδωμι, κύριος, εἰμί, ἡμ...","[εἶπον, Ἰησοῦς, υἱός, ἄνθρωπος, λέγω, κύριος, ...","[ἀποκρίνω, Ἰησοῦς, εἶπον, ἀφίημι, πρέπω, πληρό...","[(δικαιοσύνη, 7), (πᾶς, 2), (μακάριος, 2), (οὐ..."
1,tlg0031b,None,1,[New Testament - Mark],[tlg0031.tlg002.perseus-grc2.xml],11274,5298,790,{'0.5': 1},"[[ἀρχή, εὐαγγέλιον, Ἰησοῦς, Χριστός, γράφω, ἠσ...","[[ἀρχή, εὐαγγέλιον, Ἰησοῦς, Χριστός], [γράφω, ...",(),(),[],[],[],[]
2,tlg0031luke,Luke (the evangelist),2,"[New Testament - Luke, New Testament - Acts]","[tlg0031.tlg003.perseus-grc2.xml, tlg0031.tlg0...",37864,18291,2234,{'0.5': 1},"[[πολύς, ἐπιχειρέω, ἀνατάσσομαι, διήγησις, πλη...","[[πολύς, ἐπιχειρέω, ἀνατάσσομαι, διήγησις, πλη...","(ποιέω, δίδωμι, κύριος, εἰμί, τὶς, ἡμέρα, αὐτό...",(δικαιοσύνη),"[ποιέω, δίδωμι, κύριος, εἰμί, τὶς, ἡμέρα, αὐτό...",[δικαιοσύνη],"[Ζαχαρίας, πατήρ, πίμπλημι, πνεῦμα, ἅγιος, προ...","[(πᾶς, 5), (δικαιοσύνη, 5), (ἅγιος, 4), (πατήρ..."
3,tlg0031john,Johnannine literature,4,"[New Testament - John, New Testament - 1 John,...","[tlg0031.tlg004.perseus-grc2.xml, tlg0031.tlg0...",18189,8221,1339,{'0.5': 1},"[[εἰμί, ἀρχή, εἰμί, λόγος, λόγος, εἰμί, θεός, ...","[[εἰμί, ἀρχή, εἰμί, λόγος, λόγος, εἰμί, θεός, ...","(ἀγαπάω, κόσμος, πᾶς, μαθητής, ἔργον, οὗτος, π...","(λέγω, Ἰησοῦς, εἶπον, ἀποκρίνω, οὗτος, εἰμί, κ...","[ἀγαπάω, κόσμος, πᾶς, μαθητής, ἔργον, οὗτος, π...","[λέγω, Ἰησοῦς, εἶπον, ἀποκρίνω, οὗτος, εἰμί, κ...","[ἔρχομαι, ἐκεῖνος, ἐλέγχω, κόσμος, ἁμαρτία, δι...","[(δικαιοσύνη, 5), (δίκαιος, 3), (εἰμί, 3), (πο..."
4,tlg0031paul,Paul of Tarsus,7,"[New Testament - Romans, New Testament - 1 Cor...","[tlg0031.tlg006.perseus-grc2.xml, tlg0031.tlg0...",24066,11113,1632,{'0.5': 1},"[[Παῦλος, δοῦλος, Ἰησοῦς, Χριστός, κλητός, ἀπό...","[[Παῦλος, δοῦλος, Ἰησοῦς, Χριστός, κλητός, ἀπό...","(χριστός, ἔχω, νομός, δίδωμι, κύριος, ἔθνος, ε...","(Χριστός, Ἰησοῦς, κύριος, εἰμί, θεός, χάρις, π...","[χριστός, ἔχω, νομός, δίδωμι, κύριος, ἔθνος, ε...","[Χριστός, Ἰησοῦς, κύριος, εἰμί, θεός, χάρις, π...","[δικαιοσύνη, θεός, ἀποκαλύπτω, πίστις, πίστις,...","[(δικαιοσύνη, 48), (θεός, 18), (πίστις, 16), (..."
5,tlg0031pspa,Pauline literature,6,"[New Testament - Ephesians, New Testament - Co...","[tlg0031.tlg010.perseus-grc2.xml, tlg0031.tlg0...",8324,4115,390,{'0.5': 1},"[[Παῦλος, ἀπόστολος, Χριστός, Ἰησοῦς, θέλημα, ...","[[Παῦλος, ἀπόστολος, Χριστός, Ἰησοῦς, θέλημα, ...","(αἰών, πᾶς, γνωρίζω, χριστός, ἔργον, ἔχω, πνεῦ...","(Χριστός, Ἰησοῦς, κύριος, θεός, πατήρ, ἀγαθός,...","[αἰών, πᾶς, γνωρίζω, χριστός, ἔργον, ἔχω, πνεῦ...","[Χριστός, Ἰησοῦς, κύριος, θεός, πατήρ, ἀγαθός,...","[μανθάνω, χριστός, ἀκούω, διδάσκω, εἰμί, ἀλήθε...","[(δικαιοσύνη, 8), (ἀλήθεια, 4), (εἰμί, 3), (ἄν..."
6,tlg0031c,None,1,[New Testament - Hebrews],[tlg0031.tlg019.perseus-grc2.xml],4971,2512,262,{'0.5': 1},"[[πολυμερής, καί, πολύτροπος, θεός, λαλέω, πατ...","[[πολυμερής, καί, πολύτροπος, θεός, λαλέω, πατ...","(κοπεύς, ζωή, τέλος, ἑρμηνεύω, πᾶς, ἔχω, οὗτος...","(εἰμί, ἀρχιερεύς, θεός, Μελχισέδεκ, γίγνομαι, ...","[κοπεύς, ζωή, τέλος, ἑρμηνεύω, πᾶς, ἔχω, οὗτος...","[εἰμί, ἀρχιερεύς, θεός, Μελχισέδεκ, γίγνομαι, ...","[ἀγαπάω, δικαιοσύνη, μισέω, ἀνομία, μετέχω, γά...","[(δικαιοσύνη, 6), (βασιλεύς, 5), (εἰμί, 3), (π..."
7,tlg0031d,None,1,[New Testament - James],[tlg0031.tlg020.perseus-grc2.xml],1734,894,140,{'0.5': 1},"[[θεός, κύριος, Ἰησοῦς, Χριστός, δοῦλος, φυλή,...","[[θεός, κύριος, Ἰησοῦς, Χριστός, δοῦλος, φυλή,...","(φίλος, τέλειος, κατανοέω, συνεργέω, κόσμος, γ...","(ἀγ